In [4]:
# determines if a is a quadratic residue of p
def legendre(a, p, d = 2):
    symbol = ((a) ** ((p - 1) // d)) % p
    # handles negative -1
    return symbol if symbol < 2 else -1

In [18]:
# Recursive form of gcd
def gcd(a, b):
    return b if a == 0 else gcd(b%a, a)

In [166]:
## Extended Euclidean Algorithm
def ext_euclid(a, b):
    a, b = sorted((a, b % a))
    
    # remainders
    r = [b, a]
    
    # coefficient of b
    s = [1, 0]
    
    # coefficient of a
    t = [0, 1]
    
    # compute values until remainder is 0
    i = 1
    while(r[i] != 0):
        q = (r[i - 1] // r[i])
        r.append(r[i - 1] - q * r[i])
        s.append(s[i - 1] - q * s[i])
        t.append(t[i - 1] - q * t[i])
        i += 1
        
    # return relevant coefficients and remainder
    return t[i - 1] % b, s[i - 1], r[i - 1]

In [192]:
# cathode ray tu-- sorry... Chinese Remainder Theorem
# x = a_k mod n_k
def crt(a: list, n: list):
    # first, verify lists are of same size
    assert len(a) == len(n)
    
    # next, verify coprimality and generate product
    N = 1
    for i in n:
        assert (gcd(i, N) == 1)
        N *= i
        
    # now, add element N / n_i = y_i to each n
    n = [(i, N // i) for i in n]
    
    # next, add element multiplicative inverse of y_i = z_i to each n
    n = [i + tuple([ext_euclid(*i)[0]]) for i in n]

    # now, return x = sum(a * y * z) and uniqueness factor
    return sum([a[i] * n[i][1] * n[i][2] for i in range(len(n))]) % N, N

In [214]:
# Prime Factorialization
from collections import defaultdict

# Short prime finder
prime_list = [2] + [*filter(lambda i:all(i%j for j in range(3,i,2)), range(3,10000,2))] 

# Prime Factoring Algorithm
def fact(n):
    # dictionary with default value
    out = defaultdict(int)
    
    # fresh new prime list
    primes = prime_list.copy()

    f = primes.pop(0)
    while f <= n:
        if n % f == 0:
            out[f] += 1
            n //= f
        else:
            f = primes.pop(0)
    return dict(out) # [j for k in [([i] * out[i]) for i in out] for j in k]

In [201]:
# List comprension to find number of coprimes less than n
def naiive_tot(n):
    return len([i for i in range(n) if gcd(n, i) == 1])

In [202]:
# totient of a power of a prime
def p_tot(p, n):
    return (p - 1) * p ** (n - 1)

In [203]:
# finds the totient of a number using its factorialization
def smart_tot(n):
    out     = 1
    factors = fact(n)
    for i in factors:
        out *= p_tot(i, factors[i])
    return out

### 2.3

#### 2. Find all integers that satisfy simultaneously:

$x \equiv 2 (\mod 3)$, $x \equiv 3(\mod 5)$, $x \equiv 5 (\mod 2)$

In [193]:
congr = [2, 3, 5]
p     = [3, 5, 2]

# find such an integer using chinese remainder theorem
r = crt(congr, p)

# verify results and print
for i in range(len(p)):
    assert r[0] % p[i] == congr[i] % p[i]
    print("{} + {}n = {} mod {}".format(r[0], r[1], congr[i], p[i]))

23 + 30n = 2 mod 3
23 + 30n = 3 mod 5
23 + 30n = 5 mod 2


#### 4. Find all integers that give the remainders $1,2,3$ when divided by $3,4,5$, respectively.

In [194]:
congr = [1, 2, 3]
p     = [3, 4, 5]

# find such an integer using chinese remainder theorem
r = crt(congr, p)

# verify results and print
for i in range(len(p)):
    assert r[0] % p[i] == congr[i] % p[i]
    print("{} + {}n = {} mod {}".format(r[0], r[1], congr[i], p[i]))

58 + 60n = 1 mod 3
58 + 60n = 2 mod 4
58 + 60n = 3 mod 5


#### 9. For what values of $n$ is $\phi(n)$ odd?

$n > 1$ can be expressed as the product of primes, so $n=2^mp_1^{a_1} ... p_k^{a_k}$ where $a_i$ is an odd prime.

Therefore, $\phi(n)$ can be expressed as $\phi(2^m)\phi(p_1^{a_1}) ... \phi({p_k^{a_k}})$.

$\phi(p) = p^{k-1}(p-1)$ where $p$ is a prime.

$2 \nmid \phi(p^k)$ where $p$ is prime $\iff 2 \nmid p^{k-1}(p-1) \implies p = 2^1$.

Because all prime numbers except for $2$ have even totients, $2 \nmid \phi(n)  \iff 2 \nmid \phi(2^m)\phi(p_1^{a_1}) ... \phi({p_k^{a_k}}) \iff n = 2$ if $n > 1$.

By observation, $\phi(1)=1$. Also, note that multiplication by a unit does not affect the value of the totient.

Therefore, $\phi(n)$ is odd $\iff n \in {-2,-1,1,2}$.

#### 10. Find the number of positive integers $\leq 3600$ that are prime to $3600$.

In [236]:
n = 3600

# first, factor number
factors = fact(n)

# let's verify that we have the right factors
res = 1
for i in factors: res *= i ** factors[i]
assert res == n
for i in factors:
    print("({}^{})".format(i, factors[i]), end="")
print(" = {}".format(n))

# present proposition
tot = 1
for i in factors:
    print("ϕ({}^{})".format(i, factors[i]), end="")
print(" = ϕ({})".format(n))

# print results of totient of factors
for i in factors:
    tot *= (e_tot := p_tot(i, factors[i]))
    print("({})".format(e_tot), end="")
print(" = {} = ϕ({})".format(tot, n), end="")

# check with naaive approach
if (naiive_tot(n) == tot):
    print("{}✓".format(" " * 5))
else:
    print("{}:(".format(" " * 5))

(2^4)(3^2)(5^2) = 3600
ϕ(2^4)ϕ(3^2)ϕ(5^2) = ϕ(3600)
(8)(6)(20) = 960 = ϕ(3600)     ✓


### 2.6

#### 3. Solve $f(x)=x^3+x+57\equiv 0 (\mod 5 ^ 3)$

We will be using Hensel's Lemma.

Note that $f(4)=(4)^3+(4)+57=125\equiv 0 (\mod 5)$.

Additionally, $f'(4)=48+1=49$.

By Hensel's Lemma, there exists a unique solution to the equation $f(x)\equiv 0 (\mod 5^{1+2})$.

In [3]:
assert (4 ** 3 + 4 + 57) % 5 == 0
assert 3 * (4) ** 2 + 1 == 49

### 3.2

#### 2. Prove that if $p$ and $q$ are distinct primes of the form $4k+3$, and if $x^2\equiv p (\mod q))$ has no solution, then $x^2 \equiv q (\mod p)$ has two solutions. 

#### 6. Decide whether $x^2 \equiv 150 (\mod 1009)$ is solvable or not.

In [195]:
assert (139 ** 2) % 1009 == 150
assert legendre(150, 1009) == 1

139 is a solution, so it must be solvable.

Additionally, 150 is a quadratic residue of 1009, so it solvable.

#### 7. Find all primes $p$ such that $x^2 \equiv 13 (\mod p)$ has a solution.

#### 10. Of which primes is $-2$ a quadratic residue? 

$(\frac{-2}{p})$